<a href="https://colab.research.google.com/github/LuanK63/6351071043_LAB3/blob/main/6351071043_LAB3_Bai4(Trang9).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

4. (Lập	trình)	Sử	dụng Neural	Network	để	xây	dựng	mô	hình	phân	lớp	trên
bộ	dữ	liệu	Iris	từ	thư	viện	sklearn (k-fold	cross	validation):

a) Đọc	bộ	dữ	liệu	Iris	từ	sklearn

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
import tensorflow as tf

iris = load_iris()
X = iris.data
y = iris.target

b) Chuẩn	hóa	dữ liệu data về	đoạn	[0,1]

In [2]:

max_values = X.max(axis=0)
X = X / max_values

print("Kích thước dữ liệu:", X.shape)
print("Các lớp:", iris.target_names)
print("Đã chuẩn hóa dữ liệu.")

Kích thước dữ liệu: (150, 4)
Các lớp: ['setosa' 'versicolor' 'virginica']
Đã chuẩn hóa dữ liệu.


c) Định	nghĩa	hàm	create_model,	bao	gồm:
- Model	là	Neural Network	tương	tự	như	câu	1.h
- Biên	 dịch	 mô	 hình	 sử	 dụng optimizer:	 adam;	 loss:
sparse_categorical_crossentropy;	metric:	accuracy

In [3]:
def create_model():
    model = Sequential()
    model.add(Input(shape=(4,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(20, activation='relu'))
    model.add(Dense(3, activation='softmax'))

    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

d) Huấn	luyện	mô	hình	bằng	k-fold	cross	validation	(với	k=10)

Trong	quá	trình	huấn	luyện	lưu	và	in	ra	kết	quả	accuracy,	precision,	recall,	f1-
score	của	từng	fold.	Lưu	lại	mô	hình	sau	mỗi	fold.

In [4]:
print("\nNguyễn Thành Luân - 6351071043")
k = 10
kfold = KFold(n_splits=k, shuffle=True, random_state=42)

fold_accuracies = []
fold_precisions = []
fold_recalls = []
fold_f1s = []

class_metrics = {
    0: {'precision': [], 'recall': [], 'f1': []},
    1: {'precision': [], 'recall': [], 'f1': []},
    2: {'precision': [], 'recall': [], 'f1': []}
}

saved_models = []

fold_no = 1
for train_idx, val_idx in kfold.split(X):
    X_train_fold, X_val_fold = X[train_idx], X[val_idx]
    y_train_fold, y_val_fold = y[train_idx], y[val_idx]

    model = create_model()
    model.fit(X_train_fold, y_train_fold, epochs=100, batch_size=16, verbose=0)

    y_pred_prob = model.predict(X_val_fold, verbose=0)
    y_pred = np.argmax(y_pred_prob, axis=1)

    acc = accuracy_score(y_val_fold, y_pred)

    prec_per_cls, rec_per_cls, f1_per_cls, _ = precision_recall_fscore_support(y_val_fold, y_pred, average=None, zero_division=0)
    prec_avg, rec_avg, f1_avg, _ = precision_recall_fscore_support(y_val_fold, y_pred, average='weighted', zero_division=0)

    fold_accuracies.append(acc)
    fold_precisions.append(prec_avg)
    fold_recalls.append(rec_avg)
    fold_f1s.append(f1_avg)
    saved_models.append(model)

    for cls in range(3):
        if cls in np.unique(y_val_fold):
            class_metrics[cls]['precision'].append(prec_per_cls[cls])
            class_metrics[cls]['recall'].append(rec_per_cls[cls])
            class_metrics[cls]['f1'].append(f1_per_cls[cls])

    print(f"Fold {fold_no}: Acc={acc:.4f}, Precision={prec_avg:.4f}, Recall={rec_avg:.4f}, F1={f1_avg:.4f}")
    fold_no += 1



Nguyễn Thành Luân - 6351071043
Fold 1: Acc=0.9333, Precision=0.9500, Recall=0.9333, F1=0.9351
Fold 2: Acc=1.0000, Precision=1.0000, Recall=1.0000, F1=1.0000
Fold 3: Acc=1.0000, Precision=1.0000, Recall=1.0000, F1=1.0000
Fold 4: Acc=1.0000, Precision=1.0000, Recall=1.0000, F1=1.0000


Fold 5: Acc=1.0000, Precision=1.0000, Recall=1.0000, F1=1.0000


Fold 6: Acc=0.8667, Precision=0.9200, Recall=0.8667, F1=0.8700
Fold 7: Acc=0.8667, Precision=0.9000, Recall=0.8667, F1=0.8540
Fold 8: Acc=1.0000, Precision=1.0000, Recall=1.0000, F1=1.0000
Fold 9: Acc=1.0000, Precision=1.0000, Recall=1.0000, F1=1.0000
Fold 10: Acc=1.0000, Precision=1.0000, Recall=1.0000, F1=1.0000


e) In	ra	Average	Accuracy,	Average	Recall,	Average	Precision,	Average	F1-
score	của	mô	hình	sau	quá	trình	huấn	luyện với	k-fold	cross	validation

In [5]:
print("\nNguyễn Thành Luân - 6351071043")
print("--- KẾT QUẢ TRUNG BÌNH TOÀN MÔ HÌNH (AVERAGE MACRO/WEIGHTED) ---")
print(f"Average Accuracy:  {np.mean(fold_accuracies):.4f}")
print(f"Average Precision: {np.mean(fold_precisions):.4f}")
print(f"Average Recall:    {np.mean(fold_recalls):.4f}")
print(f"Average F1-score:  {np.mean(fold_f1s):.4f}")


Nguyễn Thành Luân - 6351071043
--- KẾT QUẢ TRUNG BÌNH TOÀN MÔ HÌNH (AVERAGE MACRO/WEIGHTED) ---
Average Accuracy:  0.9667
Average Precision: 0.9770
Average Recall:    0.9667
Average F1-score:  0.9659


f) In	ra	Average	Recall,	Average	Precision,	Average	F1-score	cho	từng	class
của	mô	hình	sau	quá	trình	huấn	luyện với	k-fold	cross	validation.

In [6]:
print("\nNguyễn Thành Luân - 6351071043")
print("\n--- KẾT QUẢ TRUNG BÌNH CHO TỪNG CLASS ---")
for cls in range(3):
    cls_name = iris.target_names[cls]
    avg_prec = np.mean(class_metrics[cls]['precision'])
    avg_rec = np.mean(class_metrics[cls]['recall'])
    avg_f1 = np.mean(class_metrics[cls]['f1'])

    print(f"Class '{cls_name}':")
    print(f"  - Avg Precision: {avg_prec:.4f}")
    print(f"  - Avg Recall:    {avg_rec:.4f}")
    print(f"  - Avg F1-score:  {avg_f1:.4f}")


Nguyễn Thành Luân - 6351071043

--- KẾT QUẢ TRUNG BÌNH CHO TỪNG CLASS ---
Class 'setosa':
  - Avg Precision: 1.0000
  - Avg Recall:    1.0000
  - Avg F1-score:  1.0000
Class 'versicolor':
  - Avg Precision: 1.0000
  - Avg Recall:    0.9000
  - Avg F1-score:  0.9376
Class 'virginica':
  - Avg Precision: 0.9100
  - Avg Recall:    1.0000
  - Avg F1-score:  0.9464


g) Sử	dụng	các	mô	hình	phân	lớp	có	F1-score	tốt	nhất	đã	huấn	luyện	ở	trên
cho	03	mẫu	dữ	liệu	mới	sau:
  
  [6.2,	2.9,	4.3,	1.3]

  [5.1,	3.5,	1.4,	0.2]

  [7.3,	2.8,	6.4,	2.1]

In [7]:
print("\nNguyễn Thành Luân - 6351071043")
best_fold_idx = np.argmax(fold_f1s)
best_model = saved_models[best_fold_idx]
print(f"Sử dụng model từ Fold {best_fold_idx + 1} (F1-score: {fold_f1s[best_fold_idx]:.4f})")

new_data = np.array([
    [6.2, 2.9, 4.3, 1.3],
    [5.1, 3.5, 1.4, 0.2],
    [7.3, 2.8, 6.4, 2.1]
])

new_data_norm = new_data / max_values

predictions_prob = best_model.predict(new_data_norm)
predictions = np.argmax(predictions_prob, axis=1)

print("\n--- KẾT QUẢ DỰ ĐOÁN ---")
for i, pred in enumerate(predictions):
    class_name = iris.target_names[pred]
    print(f"Mẫu {new_data[i]} -> Dự đoán: {pred} ({class_name})")


Nguyễn Thành Luân - 6351071043
Sử dụng model từ Fold 2 (F1-score: 1.0000)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step

--- KẾT QUẢ DỰ ĐOÁN ---
Mẫu [6.2 2.9 4.3 1.3] -> Dự đoán: 1 (versicolor)
Mẫu [5.1 3.5 1.4 0.2] -> Dự đoán: 0 (setosa)
Mẫu [7.3 2.8 6.4 2.1] -> Dự đoán: 2 (virginica)
